In [1]:
import pandas as pd
import os
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk import word_tokenize
from nltk.tag import CRFTagger

/var/folders/4f/xq7119ld627_2wyd6dkmwl0m0000gn/T/ipykernel_69868/2225076835.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df_anis = pd.read_csv('https://raw.github.com/ghazafm/SocialMediaSentiment/main/preprocessing/Training/data/raw/anis.csv')
df_prabowo = pd.read_csv('https://raw.github.com/ghazafm/SocialMediaSentiment/main/preprocessing/Training/data/raw/prabowo.csv')
df_ganjar = pd.read_csv('https://raw.github.com/ghazafm/SocialMediaSentiment/main/preprocessing/Training/data/raw/ganjar.csv')

df_anis.drop('Unnamed: 0',axis=1,inplace=True)
df_prabowo.drop('Unnamed: 0',axis=1,inplace=True)
df_ganjar.drop('Unnamed: 0',axis=1,inplace=True)

In [3]:
df_anis.drop_duplicates(inplace=True)
df_prabowo.drop_duplicates(inplace=True)
df_ganjar.drop_duplicates(inplace=True)
df_anis.dropna(inplace=True)
df_prabowo.dropna(inplace=True)
df_ganjar.dropna(inplace=True)

In [4]:
alay = pd.read_csv('https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv')

In [5]:
alay = dict(zip(alay['slang'], alay['formal']))

In [6]:
def cek_alay(word, alay):
    return alay.get(word, word)


def clear_alay(data):
    words = str(data)
    words = words.split()
    cleared_words = [cek_alay(word, alay) for word in words]
    return ' '.join(cleared_words)

In [7]:
df_anis['no_alay'] = df_anis['Tweet'].apply(clear_alay)
df_prabowo['no_alay'] = df_prabowo['Tweet'].apply(clear_alay)
df_ganjar['no_alay'] = df_ganjar['Tweet'].apply(clear_alay)

In [8]:
def tokenizer(text):
    text = word_tokenize(text)
    return text

In [9]:
ct = CRFTagger()
ct.set_model_file('all_indo_man_tag_corpus_model.crf.tagger')

In [10]:
tokenize_anis = df_anis['no_alay'].apply(tokenizer)
tokenize_prabowo = df_prabowo['no_alay'].apply(tokenizer)
tokenize_ganjar = df_ganjar['no_alay'].apply(tokenizer)

In [11]:
df_anis['pos'] = ct.tag_sents(tokenize_anis)
df_prabowo['pos'] = ct.tag_sents(tokenize_prabowo)
df_ganjar['pos'] = ct.tag_sents(tokenize_ganjar)

In [12]:
df_anis

,Tweet,label,no_alay,pos
0,info anies presiden,Positive,info anies presiden,"[(info, NN), (anies, NN), (presiden, NN)]"
1,politisi partai gerindra sandiaga uno menjawab...,Positive,politisi partai gerindra sandiaga uno menjawab...,"[(politisi, NN), (partai, NN), (gerindra, NN),..."
2,lanjut pak anies kita kawal sampai jadi presiden,Positive,lanjut pak anies kita kawal sampai jadi presiden,"[(lanjut, VB), (pak, NN), (anies, NN), (kita, ..."
3,semoga allah swt menyelamatkan bangsa dan nega...,Positive,semoga allah swt menyelamatkan bangsa dan nega...,"[(semoga, SC), (allah, VB), (swt, NN), (menyel..."
4,chotimah kasian ya pa anies makanya sudah teka...,Positive,chotimah kasihan ya apa anies makanya sudah te...,"[(chotimah, NN), (kasihan, NN), (ya, NN), (apa..."
...,...,...,...,...
9995,tidak ada gejolak sara selama membangun pks pu...,Negative,tidak ada gejolak sara selama membangun pks pu...,"[(tidak, NEG), (ada, VB), (gejolak, NN), (sara..."
9996,ubedilah mahfud md otak di balik perppu ciptak...,Negative,ubedilah mahfud md otak di balik perppu ciptak...,"[(ubedilah, NN), (mahfud, FW), (md, FW), (otak..."
9997,my presiden mranies,Negative,my presiden mranies,"[(my, FW), (presiden, FW), (mranies, FW)]"
9998,pa anies presiden,Negative,apa anies presiden,"[(apa, WH), (anies, NN), (presiden, NN)]"


In [13]:
double_meaning = [
    'jadi', 'menjadi', 'bapak', 'kalau', 'rakyat', 'siapa', 
    'apa', 'orang', 'bakal', 'sama', 'pasang', 'jelang', 'tahun', 'hari', 
    'bersama', 'mau', 'tetap', 'buat', 'for', 'bukan', 'semua', 
    'terus', 'si', 'inilah', 'kan', 'tak', 'banyak', 'meski', 'lebih', 'keputusan', 
    'final', 'paling', 'hasil', 'umum', 'tepat', 'tersebut', 'total', 'klik', 'capres', 
    'pilih', 'pemilihan', 'terpilih', 'survei', 'survey', 'pemilu', 'terkait', 'fahnoor', 
    'nan', 'calon', 'pilpres', 'resmi', 'cocok', 'politik', 'ribuan', 'ratusan', 'nama','maju',
    'hut', 'dapat', 'semoga', 'beliau', 'besar', 'makin', 'layak', 'partai', 'mendukung', 'dukung', 
    'dukungan', 'gubernur', 'masyarakat', 'warga','presiden','ri','inismyname','pilpres','nan','calon','indonesia','survei','survey','pemilu'
]

name = [
    'inismyname', 'indonesia', 'rosiade', 'joko', 'jokowi', 'widodo', 'ridwan', 'kamil', 'rosiade', 
    'thohir', 'mujani', 'erick', 'saiful', 'chotimah', 'ahy', 'bukan', 'aniesahy', 'ahmad', 
    'pks', 'pdip', 'jawa', 'puan', 'maharani', 'pan', 'jateng', 'tengah', 'megawati', 'ppp', 
    'rasyid', 'gerindra', 'nasdem', 'demokrat', 'pkb', 'allah', #maaffff,
    'anis', 'anies', 'baswedan', 'prabowo', 'subianto', 'ganjar', 'pranowo','fahnoor', 'amien','sandiaga',
    'chotimah', 'uno'
]

def clean_manual(data,token=False):
    temp = []
    if token:
        for tup in data:
            if tup[0] in double_meaning or tup[0] in name:
                continue
            temp.append(tup)
        return temp
    else:
        for tup in data:
            if tup[0] in double_meaning or tup[0] in name:
                continue
            temp.append(tup[0])
            temp.append(tup[1])
    temp = ' '.join(temp)
    return temp

In [14]:
df_anis['clean_manual'] = df_anis['pos'].apply(clean_manual)
df_prabowo['clean_manual'] = df_prabowo['pos'].apply(clean_manual)
df_ganjar['clean_manual'] = df_ganjar['pos'].apply(clean_manual)


tokenize_anis = df_anis['pos'].apply(clean_manual,token=True)
tokenize_prabowo = df_prabowo['pos'].apply(clean_manual,token=True)
tokenize_ganjar = df_ganjar['pos'].apply(clean_manual,token=True)

In [15]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
def stemming(data,token=False):
   temp = []
   last = ''
   for tup in data:
      tup_temp = stemmer.stem(tup[0])
      don = [tup_temp,tup[1]]
      if token:
         temp.append(don)
         last = temp
      else:
         temp.append(' '.join(don))
         last = ' '.join(temp)
   return last

In [16]:
df_anis['stemmed'] = tokenize_anis.apply(stemming)
df_prabowo['stemmed'] = tokenize_prabowo.apply(stemming)
df_ganjar['stemmed'] = tokenize_ganjar.apply(stemming)


tokenize_anis = tokenize_anis.apply(stemming,token=True)
tokenize_prabowo = tokenize_prabowo.apply(stemming,token=True)
tokenize_ganjar = tokenize_ganjar.apply(stemming,token=True)

In [17]:
stopword = StopWordRemoverFactory().create_stop_word_remover().dictionary.words

def clear_stopwords(data,token=False):
    temp = []
    last = ''
    for tup in data:
        if(tup[0] in stopword):
            continue
        if token:
            temp.append(tup)
            last = temp
        else:
            temp.append(' '.join(tup))
            last = ' '.join(temp)
    return last

In [38]:
df_anis['no_stopwords'] = tokenize_anis.apply(clear_stopwords)
df_prabowo['no_stopwords'] = tokenize_prabowo.apply(clear_stopwords)
df_ganjar['no_stopwords'] = tokenize_ganjar.apply(clear_stopwords)


tokenize_anis = tokenize_anis.apply(clear_stopwords,token=True)
tokenize_prabowo = tokenize_prabowo.apply(clear_stopwords,token=True)
tokenize_ganjar = tokenize_ganjar.apply(clear_stopwords,token=True)

In [54]:
def removena(data):
    if len(data) == 0:
        return None
    return data

In [55]:
df_anis['no_stopwords'] = df_anis['no_stopwords'].apply(removena)
df_prabowo['no_stopwords'] = df_prabowo['no_stopwords'].apply(removena)
df_ganjar['no_stopwords'] = df_ganjar['no_stopwords'].apply(removena)

In [57]:
df_anis.dropna(inplace=True)
df_prabowo.dropna(inplace=True)
df_ganjar.dropna(inplace=True)

In [60]:
concat = pd.concat([df_anis,df_prabowo,df_ganjar])

In [ ]:
dir = os.path.abspath(os.path.join('..', 'data/clean/pos_tagging'))
df_anis.to_csv(f'{dir}/anis.csv',index=False)
df_prabowo.to_csv(f'{dir}/prabowo.csv',index=False)
df_ganjar.to_csv(f'{dir}/ganjar.csv',index=False)
concat.to_csv(f'{dir}/gabungan.csv',index=False)